In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torchvision.datasets as dset
import torchvision.utils as vutils
from torchvision.utils import make_grid
from PIL import Image
import os
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from CustomDataset import CustomDataset
from torch import optim
from tqdm.auto import tqdm
import csv

In [ ]:
dataroot = os.path.relpath("DATA/TIBET")
dataroot

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
ngpu = 1
criterion = nn.BCEWithLogitsLoss()
n_epochs = 5000 #@param
z_dim = 64 #@param
display_step = 500 #@param
batch_size = 64 #@param
lr = 0.00001 #@param
outf = os.path.relpath("OUTPUT/DCGAN")

In [ ]:
tf = T.Compose([
    T.Resize(64),
    T.Grayscale(),
    T.ToTensor(),
    T.Normalize((0.5),(0.5))])
tibet = CustomDataset(dataroot=dataroot, transform=tf)
print(f"train size: {len(tibet)}")

In [ ]:
dataloader = DataLoader(tibet, batch_size, True, num_workers=2)

In [ ]:
def get_generator_block(input_dim, output_dim):
    '''
    Function for returning a block of the generator's neural network
    given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a generator neural network layer, with a linear transformation 
          followed by a batch normalization and then a relu activation
    '''
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True),
    )


In [ ]:
class Generator(nn.Module):
    '''
    Generator Class
    Values:
        z_dim: the dimension of the noise vector. int-like
        im_dim: the dimension of the images, fitted for the dataset used. int-like
          (MNIST images are 28 x 28 = 784 so that is your default)
        hidden_dim: the inner dimension. int-like
    '''
    def __init__(self, z_dim=10, im_dim=4096, hidden_dim=128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            nn.Linear(hidden_dim*8,im_dim),
            nn.Sigmoid()
        )
    def forward(self, noise):
        '''
        Function for completing a forward pass of the generator: Given a noise tensor, 
        returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        '''
        return self.gen(noise)
    
    def get_gen(self):
        '''
        Returns:
            the sequential model
        '''
        return self.gen

In [ ]:
def get_noise(n_samples, z_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim),
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    '''
    noise = torch.normal(mean=0.0,std=1,size=(n_samples, z_dim)).to(device)
    return noise

In [ ]:
def get_discriminator_block(input_dim, output_dim):
    '''
    Discriminator Block
    Function for returning a neural network of the discriminator given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a discriminator neural network layer, with a linear transformation 
          followed by an nn.LeakyReLU activation with negative slope of 0.2 
    '''
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(negative_slope=0.2,inplace=True)
    )

In [ ]:
class Discriminator(nn.Module):
    '''
    Discriminator Class
    Values:
        im_dim: the dimension of the images, fitted for the dataset used, a scalar
            (MNIST images are 28x28 = 784 so that is your default)
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, im_dim=4096, hidden_dim=128):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim,1)
        )

    def forward(self, image):
        '''
        Function for completing a forward pass of the discriminator: Given an image tensor, 
        returns a 1-dimension tensor representing fake/real.
        Parameters:
            image: a flattened image tensor with dimension (im_dim)
        '''
        return self.disc(image)
    
    def get_disc(self):
        '''
        Returns:
            the sequential model
        '''
        return self.disc

In [ ]:
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
disc = Discriminator().to(device) 
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)

In [ ]:
def get_disc_loss(gen, disc, criterion, real, num_images, z_dim, device):
    '''
    Return the loss of the discriminator given inputs.
    Parameters:
        gen: the generator model, which returns an image given z-dimensional noise
        disc: the discriminator model, which returns a single-dimensional prediction of real/fake
        criterion: the loss function, which should be used to compare 
               the discriminator's predictions to the ground truth reality of the images 
               (e.g. fake = 0, real = 1)
        real: a batch of real images
        num_images: the number of images the generator should produce, 
                which is also the length of the real images
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    Returns:
        disc_loss: a torch scalar loss value for the current batch
    '''
    # Calculate loss for fake image
    noise = get_noise(num_images, z_dim, device=device)
    x_fake, y_fake = gen(noise).detach(),torch.zeros(num_images, 1, device=device)
    disc_fake_score = disc(x_fake)
    disc_fake_loss = criterion(disc_fake_score, y_fake)
    
    # Calculate loss for real image
    disc_real_score = disc(real)
    disc_real_loss = criterion(disc_real_score, torch.ones(num_images,1,device=device))

    disc_loss = (disc_fake_loss+disc_real_loss)/2
    return disc_loss

In [ ]:
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    '''
    Return the loss of the generator given inputs.
    Parameters:
        gen: the generator model, which returns an image given z-dimensional noise
        disc: the discriminator model, which returns a single-dimensional prediction of real/fake
        criterion: the loss function, which should be used to compare 
               the discriminator's predictions to the ground truth reality of the images 
               (e.g. fake = 0, real = 1)
        num_images: the number of images the generator should produce, 
                which is also the length of the real images
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    Returns:
        gen_loss: a torch scalar loss value for the current batch
    '''

    noise = get_noise(num_images, z_dim).to(device)
    fake_images = gen(noise).to(device)
    disc_score_fake = disc(fake_images)
    gen_loss = criterion(disc_score_fake, torch.ones(num_images,1,device=device))

    return gen_loss

In [ ]:
def show_tensor_images(image_tensor, num_images=25, size=(1, 64, 64)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in a uniform grid.
    '''
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [ ]:
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
test_generator = True 
gen_loss = False
error = False
header = True
torch.cuda.synchronize(device)
for epoch in range(n_epochs):
    for real in tqdm(dataloader):
        cur_batch_size = len(real)

        # Flatten the batch of real images from the dataset
        real = real.view(cur_batch_size, -1).to(device)

        # Update discriminator 
        disc_opt.zero_grad()

        # Calculate discriminator loss
        disc_loss = get_disc_loss(gen, disc, criterion, real, cur_batch_size, z_dim, device)

        # Update gradients
        disc_loss.backward(retain_graph=True)

        # Update optimizer
        disc_opt.step()

        # For testing purposes, to keep track of the generator weights
        if test_generator:
            old_generator_weights = gen.gen[0][0].weight.detach().clone()

        # Update Generator
        gen.zero_grad()

        # Calculate generator loss
        gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)

        # Update gradient in Generator
        gen_loss.backward()

        # Update optimizer
        gen_opt.step()

        # Keep track of the average discriminator loss
        mean_discriminator_loss += disc_loss.item() / display_step

        # Keep track of the average generator loss
        mean_generator_loss += gen_loss.item() / display_step

        ### Visualization training ###
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Epoch {epoch}, step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            fake = gen(fake_noise)
            show_tensor_images(fake)
            show_tensor_images(real)
            
            with open("report-gan.csv", "a") as f:
                writer = csv.DictWriter(f,fieldnames=["epoch","cur_step","loss","model"])
                if header:
                    writer.writeheader()
                    writer.writerow({"epoch":epoch,"cur_step":cur_step,"loss":mean_generator_loss,"model":"generator"})    
                    writer.writerow({"epoch":epoch,"cur_step":cur_step,"loss":mean_discriminator_loss,"model":"discriminator"})
                    header = False
                else:
                    writer.writerow({"epoch":epoch,"cur_step":cur_step,"loss":mean_generator_loss,"model":"generator"})    
                    writer.writerow({"epoch":epoch,"cur_step":cur_step,"loss":mean_discriminator_loss,"model":"discriminator"})
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1